In [ ]:
!pip install lifetimes

In [ ]:
!pip install psycopg2-binary

In [ ]:
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
#import numpy as np
from lifetimes import BetaGeoFitter,GammaGammaFitter

#from sklearn.metrics import mean_squared_error
#from math import sqrt
from sklearn.preprocessing import MinMaxScaler
#from datetime import date
import datetime
import psycopg2
from datetime import date



def start_cltv():

        con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cgzkthavydhk.us-east-2.redshift.amazonaws.com', 
        port= '5439', user= 'mantix', password= 'Mantix123!')
        cur = con.cursor()

        orders_data = pd.read_sql("select * from cdp.orders_data;",con)

        orders_data=orders_data[(orders_data['tenant_id']=='TNB00001') & (orders_data['store_id']=='STOM000000001')]

        orders_data = orders_data[~orders_data['status'].isin(['Abandoned','Errored','Pending','PendingReview','Cancelled','Null'])]

        items_data = pd.read_sql("select * from cdp.items_data;",con)

        orders_data.order_number = orders_data.order_number.astype(int).astype(str)
        items_data.order_number = items_data.order_number.astype(int).astype(str)

        ord_cust = orders_data[['customer_account_id','fulfillment_date','order_number']]
        items_df = items_data[['product_name','product_code','total','order_number','quantity']]

        df =pd.merge(ord_cust,items_data[['product_name','product_code','total','order_number','quantity']],on=['order_number'],how='inner')

        df = df.drop_duplicates()

        #df = df.dropna(subset=['customer_account_id','total','fulfillment_date'])

        column_dict = {'order_number':'Invoice','product_code':'StockCode',
                'quantity':'Quantity','fulfillment_date':'InvoiceDate',
                'customer_account_id':'CustomerId','total':'TotalPrice'}

        req_col = ['Invoice','InvoiceDate','CustomerId','TotalPrice']

        df = df.rename(columns=column_dict)
        df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

        df = df.fillna(df.mean())

        today_date = datetime.datetime.now().time()
        cltv_df = df.groupby('CustomerId').agg({'InvoiceDate': [lambda date: (date.max() - date.min()).days,
                                                                lambda date: (pd.to_datetime('now')- date.min()).days],
                                        'Invoice': lambda num: num.nunique(),
                                        'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

        cltv_df.columns = cltv_df.columns.droplevel(0)
        cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']

        #Expressing "monetary value" as average earnings per purchase
        cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]
        # selection of monetary values greater than zero
        cltv_df = cltv_df[cltv_df["monetary"] > 0]
        # Expression of "recency "and "T" in weekly terms
        cltv_df["recency"] = cltv_df["recency"] / 7
        cltv_df["T"] = cltv_df["T"] / 7
        #frequency must be greater than 1.
        cltv_df = cltv_df[(cltv_df['frequency'] > 1)]
        cltv_df = cltv_df[cltv_df.recency < cltv_df['T']]

        print(cltv_df.isna().sum())
        #cltv_df.index = cltv_df.index.astype(float)
        print('cltv : ',cltv_df.head())
        cltv_df['CustomerId'] = cltv_df.index
        cltv_df['env'] = 'prod'


        ### Starting cltv model
        cltv_prod = cltv_df.copy()
        cltv_dev = pd.read_sql('SELECT * FROM cdp.cltv_support;',con)
        cltv_dev = cltv_dev.rename(columns={'t':'T','customerid':'CustomerId'})
        cltv_df = pd.concat([cltv_prod,cltv_dev])

        bgf = BetaGeoFitter(penalizer_coef=0.001)
        #setting up the model
        bgf.fit(cltv_df['frequency'],
                cltv_df['recency'],
                cltv_df['T'])

        cltv_df["expected_purc_6_month"] = bgf.predict(4*6,
                                                cltv_df['frequency'],
                                                cltv_df['recency'],
                                                cltv_df['T'])
        cltv_df["expected_purc_12_month"] = bgf.predict(4*12,
                                                cltv_df['frequency'],
                                                cltv_df['recency'],
                                                cltv_df['T'])
        cltv_df["expected_purc_24_month"] = bgf.predict(4*24,
                                                cltv_df['frequency'],
                                                cltv_df['recency'],
                                                cltv_df['T'])

        cltv_df.index = cltv_df.CustomerId

        ggf = GammaGammaFitter(penalizer_coef=0.01)
        ggf.fit(cltv_df['frequency'], cltv_df['monetary'])

        cltv_6_months = ggf.customer_lifetime_value(bgf,
        cltv_df['frequency'],
        cltv_df['recency'],
        cltv_df['T'],
        cltv_df['monetary'],
        time=6, # 6 Months
        freq="W", # Frequency of T ,in this case it is 'weekly'
        discount_rate=0.01)

        cltv_12_months = ggf.customer_lifetime_value(bgf,
        cltv_df['frequency'],
        cltv_df['recency'],
        cltv_df['T'],
        cltv_df['monetary'],
        time=12, # 6 Months
        freq="W", # Frequency of T ,in this case it is 'weekly'
        discount_rate=0.01)

        cltv_24_months = ggf.customer_lifetime_value(bgf,
        cltv_df['frequency'],
        cltv_df['recency'],
        cltv_df['T'],
        cltv_df['monetary'],
        time=24, # 6 Months
        freq="W", # Frequency of T ,in this case it is 'weekly'
        discount_rate=0.01)

        scaler = MinMaxScaler(feature_range=(0, 1)) 

        cltv_6_months = cltv_6_months.reset_index()
        cltv_6_months = cltv_6_months.rename(columns={'clv':'cltv_6M'})
        cltv_6_months['scaled_cltv_6M'] = scaler.fit_transform(cltv_6_months[['cltv_6M']])

        cltv_12_months = cltv_12_months.reset_index()
        cltv_12_months = cltv_12_months.rename(columns={'clv':'cltv_12M'})
        cltv_12_months['scaled_cltv_12M'] = scaler.fit_transform(cltv_12_months[['cltv_12M']])

        cltv_24_months = cltv_24_months.reset_index()
        cltv_24_months = cltv_24_months.rename(columns={'clv':'cltv_24M'})
        cltv_24_months['scaled_cltv_24M'] = scaler.fit_transform(cltv_24_months[['cltv_24M']])


        cltv_df = cltv_df.reset_index(drop=True)


        cltv_final = cltv_df.merge(cltv_6_months, on="CustomerId", how="inner")
        cltv_final = cltv_final.merge(cltv_12_months, on="CustomerId", how="inner")
        cltv_final = cltv_final.merge(cltv_24_months, on="CustomerId", how="inner")

        cltv_final = cltv_final[cltv_final.CustomerId.isin(cltv_prod.CustomerId)]

        req_col = ['CustomerId',
                'cltv_6M',
                'cltv_12M',
                'cltv_24M',]

        cltv_final=cltv_final[req_col]

        cltv_final = cltv_final.groupby(['CustomerId']).median().reset_index()
        cltv_final['tenant_id'] = 'TNB00001'
        cltv_final['store_id'] = 'STOM000000001'
        today_date = date.today()
        cltv_final['create_dms'] = today_date
        con.close()
        return cltv_final

def load_db(cltv_final):
    con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cgzkthavydhk.us-east-2.redshift.amazonaws.com', 
    port= '5439', user= 'mantix', password= 'Mantix123!')
    cur = con.cursor()

    def drop_table(conn,table):
        try:
            query = f"""DROP TABLE IF EXISTS {table}"""
            cur = conn.cursor()
            cur.execute(query)
            conn.commit()
            print(f'Drop table {table} successfully')
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cur.close()
            #return 1

    def create_table(conn,table):
        try:
            query_col = """(customer_account_id VARCHAR(1000),
                cltv_6M FLOAT,cltv_12M FLOAT,
                cltv_24M FLOAT,tenant_id VARCHAR(1000),store_id VARCHAR(1000),create_dms VARCHAR(1000))"""
            query = f"""
                    CREATE TABLE {table} 
                    """+query_col
            cur = conn.cursor()
            cur.execute("ROLLBACK")
            conn.commit()
            cur.execute(query)
            conn.commit()
            print(f'Successfully created the table {table}')
            cur.close()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cur.close()
            return 1
            
    def single_insert(conn, insert_req):
        """ Execute a single INSERT request """
        cursor = conn.cursor()
        try:
            cursor.execute(insert_req)
            conn.commit()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cursor.close()
            return 1
        cursor.close()
        
    def batch_insert(conn, batch_df):
        # Create a cursor object from the connection
        cursor = conn.cursor()
        statment= f"""INSERT into {table}(customer_account_id,cltv_6M,cltv_12M,cltv_24M,tenant_id,store_id,create_dms) values(%s,%s,%s,%s,%s,%s,%s);"""
        try:
            # Execute the batch INSERT request
            cursor.executemany(statment, batch_df)
            # Commit the changes to the database
            conn.commit()
            print("Batch INSERT successful.")
        except (Exception, psycopg2.DatabaseError) as error:
            # Rollback the transaction in case of an error
            conn.rollback()
            print("Batch INSERT failed, Error:", error)
        finally:
            # Close the cursor
            cursor.close() 
        
        
    def insert_dataframe(final_df,conn,table):
        batch_df=[]
        for i in final_df.index:
            current_tuple=(
                        cltv_final.loc[i]['CustomerId'], 
                        cltv_final.loc[i]['cltv_6M'],
                        cltv_final.loc[i]['cltv_12M'],
                        cltv_final.loc[i]['cltv_24M'],
                        str(cltv_final.loc[i]['tenant_id']),
                        str(cltv_final.loc[i]['store_id']),
                        str(cltv_final.loc[i]['create_dms'])
                )
            batch_df.append(current_tuple)
            if (len(batch_df) == 500): 
                batch_insert(conn, batch_df)
                batch_df=[]
                print("Current index: {}, time: {}".format(i, datetime.datetime.now()))
            #endif
        # Insert any remaining rows
        if (len(batch_df) > 0):
            batch_insert(conn,batch_df)
        print('Successfully inserted dataframe into the table')
        
    table = 'cdp.cltv'
    drop_table(con,table)
    create_table(con,table)
    startTime = datetime.datetime.now()
    insert_dataframe(cltv_final,con,table)
    print("Start: {}, Finish: {}".format(startTime, datetime.datetime.now()))

if __name__ == '__main__':

    # start cltv
    cltv_final = start_cltv()
    print(cltv_final.shape)

    # load data into database
    load_db(cltv_final)


In [ ]:
# def load_db(cltv_final):
#     con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cgzkthavydhk.us-east-2.redshift.amazonaws.com', 
#     port= '5439', user= 'mantix', password= 'Mantix123!')
#     cur = con.cursor()

#     def drop_table(conn,table):
#         try:
#             query = f"""DROP TABLE IF EXISTS {table}"""
#             cur = conn.cursor()
#             cur.execute(query)
#             conn.commit()
#             print(f'Drop table {table} successfully')
#             cur.close()
#         except (Exception, psycopg2.DatabaseError) as error:
#             print("Error: %s" % error)
#             conn.rollback()
#             cur.close()
#             #return 1

#     def create_table(conn,table):
#         try:
#             query_col = """(customer_account_id VARCHAR(1000),
#                 cltv_6M FLOAT,cltv_12M FLOAT,
#                 cltv_24M FLOAT,tenant_id VARCHAR(1000),store_id VARCHAR(1000),create_dms VARCHAR(1000))"""
#             query = f"""
#                     CREATE TABLE {table} 
#                     """+query_col
#             cur = conn.cursor()
#             cur.execute("ROLLBACK")
#             conn.commit()
#             cur.execute(query)
#             conn.commit()
#             print(f'Successfully created the table {table}')
#             cur.close()
#         except (Exception, psycopg2.DatabaseError) as error:
#             print("Error: %s" % error)
#             conn.rollback()
#             cur.close()
#             return 1
            
#     def single_insert(conn, insert_req):
#         """ Execute a single INSERT request """
#         cursor = conn.cursor()
#         try:
#             cursor.execute(insert_req)
#             conn.commit()
#         except (Exception, psycopg2.DatabaseError) as error:
#             print("Error: %s" % error)
#             conn.rollback()
#             cursor.close()
#             return 1
#         cursor.close()
        
#     def insert_dataframe(cltv_final,conn,table):
#         for i in cltv_final.index:
#             query = f"""
#                     INSERT into {table}(customer_account_id,cltv_6M,cltv_12M,cltv_24M,tenant_id,store_id,create_dms) 
#                         values('%s',%s,%s,%s,%s,%s,%s);
#                     """ % (cltv_final.loc[i]['CustomerId'], 
#                         cltv_final.loc[i]['cltv_6M'],
                        
#                         cltv_final.loc[i]['cltv_12M'],
                        
#                         cltv_final.loc[i]['cltv_24M'],
                        
#                         "'"+str(cltv_final.loc[i]['tenant_id'])+"'",
#                         "'"+str(cltv_final.loc[i]['store_id'])+"'",
#                          "'"+str(cltv_final.loc[i]['create_dms'])+"'")
#             if i%50==0:
#                 print(i,end=' ')
#             single_insert(conn, query)
#         conn.commit()
#         print('\nSuccessfully inserted dataframe into the table')
        
#     table = 'cdp.cltv'
#     drop_table(con,table)
#     create_table(con,table)
#     insert_dataframe(cltv_final,con,table)